In [ ]:
# !pip install pororo -q

+ [ ] 📄매장 LIst 확보
+ [ ] ooo

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from datetime import datetime, timedelta
import time 
import json
# from tqdm import trange

In [2]:
# URL 예시
main = 'https://map.naver.com/v5/search/크리스탈%20제이드'
store1 = 'https://map.naver.com/v5/api/sites/summary/1271525551?lang=ko' #json
store2 = 'https://pcmap.place.naver.com/restaurant/1271525551/home' #html

#### 📄매장 LIst(code) 확보

In [ ]:
# Function dependencies, for example:
# package>=version
pandas>=0.25.3
requests>=2.22.0
google-cloud-bigquery>=1.18.0
pandas_gbq>=0.17.9
beautifulsoup4>=4.4.0

In [9]:
# main.py
from datetime import datetime, timedelta
import pandas as pd
import requests
from google.cloud import bigquery
import pandas_gbq
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import json
import re

# 저장 위치 설정
project = 'boxwood-chassis-356100'
client = bigquery.Client(project)
dataset_id = 'review'
table_id = 'dinning_review_summary'
table_name = dataset_id +'.' +table_id


search_keyword = ['크리스탈제이드', '더%20키친%20일뽀르노']
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

code_list = []
# for text in search_keyword:
for search_ in search_keyword:
    url = 'https://m.map.naver.com/search2/search.naver?query=' + search_ + '&sm=clk&style=v5&page=1&displayCount=75&type=SITE_1'
    response_main = requests.get(url, headers = headers)
    # bsObj = BeautifulSoup(response_main.text, "html.parser")
    # val = bsObj('script', {"type" : "text/javascript"})[-1].text
    json_object = json.loads(re.findall('\n\tvar searchResult = (.*?);\n\tvar ',response_main.text )[0].strip())
    code_ = list(pd.DataFrame(json_object['site']['list'])['id'] )
    code_list += code_



def get_data(code_):
    date_   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    df = pd.DataFrame()
    for store_code in code_:
        store_code = store_code.replace('s', '')
        url_temp2  = 'https://pcmap.place.naver.com/restaurant/'+str(store_code)+'/home'
        response   = requests.get(url_temp2, headers = headers)
        response.encoding = 'utf-8'
        bsObj = BeautifulSoup(response.text, "html.parser")
        store_nm = bsObj('div', {'class' : 'place_didmount'})[0]('h1')[0].text
        # review_score = 0
        try:
            review_score = re.findall("별점</span><em>(.*?)</em>/5", str(bsObj('div', {'class' : 'place_didmount'})[0]))[0]
        except:
            review_score = 0
        try:
            review_count = bsObj('div', {'class' : 'place_didmount'})[0]('a', {'class' : 'place_bluelink'})[0]('em')[0].text.replace(',', '')
        except:
            review_count = 0
        try:
            blog_count   = bsObj('div', {'class' : 'place_didmount'})[0]('a', {'class' : 'place_bluelink'})[1]('em')[0].text.replace(',', '')
        except:
            blog_count = 0
        rows_to_insert = [( str(store_code), store_nm, float(review_score), int(review_count), int(blog_count), date_)]
        df = df.append(rows_to_insert)
    df = df.rename(columns={0: "STORE_CODE", 1: "STORE_NAME", 2: "REVIEW_SCORE",  3: "REVIEW_CNT",  4: "BLOG_CNT", 5: "REG_DTM"})
    return df

def data_insert(t1):
    dataset = get_data(code_list)
    dataset['REVIEW_SCORE'] = dataset['REVIEW_SCORE'].astype(float)
    dataset['REVIEW_CNT'] = dataset['REVIEW_CNT'].astype(float)
    dataset['BLOG_CNT'] = dataset['BLOG_CNT'].astype(float)
    dataset['REG_DTM'] = pd.to_datetime(dataset['REG_DTM'], format='%Y-%m-%d')
    # pandas_gbq.to_gbq(dataset, table_name, project_id=project, if_exists = 'append' )
    # return f'Hello World!'
    return dataset


In [8]:
data_insert('')

,STORE_CODE,STORE_NAME,REVIEW_SCORE,REVIEW_CNT,BLOG_CNT,REG_DTM
0,31378239,크리스탈제이드 소공점,4.21,1098.0,617.0,2023-02-14
0,1110186348,크리스탈제이드 광화문상하이팰리스점,4.30,816.0,190.0,2023-02-14
0,1004452077,크리스탈제이드 신세계백화점 본점,0.00,3.0,53.0,2023-02-14
0,1271525551,크리스탈제이드 용산 아이파크몰점,4.33,1566.0,437.0,2023-02-14
0,19882607,크리스탈제이드 현대 압구정본점,4.37,701.0,194.0,2023-02-14
0,1141410734,크리스탈제이드 상하이 팰리스 잠원점,4.43,449.0,161.0,2023-02-14
0,35831858,크리스탈제이드 상하이 팰리스 잠실 에비뉴엘점,4.28,848.0,651.0,2023-02-14
0,1121084288,크리스탈제이드 코리아 본사 사무실,4.35,513.0,0.0,2023-02-14
0,1655422043,크리스탈제이드 현대백화점 천호점,0.00,68.0,64.0,2023-02-14
0,1236473153,크리스탈제이드 AK 분당점,4.27,434.0,147.0,2023-02-14


In [ ]:
# 매장정보 수집
code = ['1271525551', '31378239',   '1110186348', '1141410734', '36220114',   '19882607', 
        '35831858',   '1655422043', '1236473153', '48175941',   '1371827794', '38230223', '1941012826',
        '1811388034', '1750908039', '1129408372', '1613879707', '1235447371', '1027299193'
        ]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
date_   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
bsObj = BeautifulSoup(response.text, "html.parser")
    
for store_code in code:
    url_temp2 = 'https://pcmap.place.naver.com/restaurant/'+str(store_code)+'/home'
    response = requests.get(url_temp2, headers = headers)
    response.encoding = 'utf-8'
    bsObj = BeautifulSoup(response.text, "html.parser")
    store_nm = bsObj('div', {'class' : 'place_didmount'})[0]('h1')[0].text
    # review_score = 0
    try:
        review_score = re.findall("별점</span><em>(.*?)</em>/5", str(bsObj('div', {'class' : 'place_didmount'})[0]))[0]
    except:
        review_score = 0
    review_count = bsObj('div', {'class' : 'place_didmount'})[0]('a', {'class' : 'place_bluelink'})[0]('em')[0].text.replace(',', '')
    blog_count   = bsObj('div', {'class' : 'place_didmount'})[0]('a', {'class' : 'place_bluelink'})[1]('em')[0].text.replace(',', '')
    rows_to_insert = [( str(store_code), store_nm, float(review_score), int(review_count), int(blog_count), date_)]
    print(rows_to_insert)

[('1271525551', '크리스탈제이드 용산 아이파크몰점', 4.32, 1342, 366, '2022-09-01 00:00:00')]
[('31378239', '크리스탈제이드 소공점', 4.2, 986, 562, '2022-09-01 00:00:00')]
[('1110186348', '크리스탈제이드 광화문상하이팰리스점', 4.3, 689, 140, '2022-09-01 00:00:00')]
[('1141410734', '크리스탈제이드 상하이 팰리스 잠원점', 4.43, 376, 134, '2022-09-01 00:00:00')]
[('36220114', '크리스탈제이드 파미에스테이션점', 4.3, 1076, 579, '2022-09-01 00:00:00')]
[('19882607', '크리스탈제이드 현대 압구정본점', 4.36, 608, 178, '2022-09-01 00:00:00')]
[('35831858', '크리스탈제이드 상하이 팰리스 잠실 에비뉴엘점', 4.29, 697, 586, '2022-09-01 00:00:00')]
[('1655422043', '크리스탈제이드 현대백화점 천호점', 0.0, 37, 30, '2022-09-01 00:00:00')]
[('1236473153', '크리스탈제이드 AK 분당점', 4.29, 301, 114, '2022-09-01 00:00:00')]
[('48175941', '크리스탈제이드 신세계 대구점', 4.44, 951, 409, '2022-09-01 00:00:00')]
[('1371827794', '크리스탈제이드 롯데 부산서면점', 4.54, 582, 218, '2022-09-01 00:00:00')]
[('38230223', '크리스탈제이드 신세계센텀시티몰점', 4.29, 600, 302, '2022-09-01 00:00:00')]
[('1941012826', '크리스탈제이드 부산 피아크점', 0.0, 111, 119, '2022-09-01 00:00:00')]
[('1811388034', '더 키친 

#### 🅰제품 리뷰 수집

In [ ]:
code = ['1271525551', '31378239',   '1110186348', '1141410734', '36220114',   '19882607', 
        '35831858',   '1655422043', '1236473153', '48175941',   '1371827794', '38230223', '1941012826',
        '1811388034', '1750908039', '1129408372', '1613879707', '1235447371', '1027299193'
        ]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}

def navermap_review(t1):
    # client = bigquery.Client()
    # project = 'thermal-rain-234004'
    # dataset_id = 'review'
    # table_id = 'review_all'
    # table_ref = client.dataset(dataset_id).table(table_id)
    # Table = client.get_table(table_ref)  # API request
    date_   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    for code_value in code:
        url_temp = 'https://map.naver.com/v5/api/sites/summary/'+str(code_value)+'?lang=ko'
        response = requests.get(url_temp).json()
        store_nm = response['name']

        url_temp2 = 'https://pcmap.place.naver.com/restaurant/'+str(code_value)+'/home'
        response = requests.get(url_temp2)
        bsObj = BeautifulSoup(response.text, "html.parser")
        if len(bsObj('span', {'class' : '_1Y6hi'})) == 3:
            review_score = float(bsObj('span', {'class' : '_1Y6hi'})[0]('em')[0].text)
            review_count = int(bsObj('span', {'class' : '_1Y6hi'})[1]('em')[0].text.replace(',', ''))
            blog_count   = int(bsObj('span', {'class' : '_1Y6hi'})[2]('em')[0].text.replace(',', ''))
        else:
            review_score = 0
            review_count = int(bsObj('span', {'class' : '_1Y6hi'})[0]('em')[0].text.replace(',', ''))
            blog_count   = int(bsObj('span', {'class' : '_1Y6hi'})[1]('em')[0].text.replace(',', ''))
        rows_to_insert = [(store_nm, code_value, review_score,  review_count, blog_count, date_)]
        print(rows_to_insert)
        # client.insert_rows(Table, rows_to_insert)

In [ ]:
navermap_review('')

[('크리스탈제이드 용산 아이파크몰점', '1271525551', 4.32, 1328, 359, '2022-08-23 00:00:00')]
[('크리스탈제이드 소공점', '31378239', 4.2, 975, 561, '2022-08-23 00:00:00')]
[('크리스탈제이드 광화문상하이팰리스점', '1110186348', 4.3, 676, 138, '2022-08-23 00:00:00')]
[('크리스탈제이드 상하이 팰리스 잠원점', '1141410734', 4.42, 377, 134, '2022-08-23 00:00:00')]
[('크리스탈제이드 파미에스테이션점', '36220114', 4.3, 1071, 573, '2022-08-23 00:00:00')]
[('크리스탈제이드 현대 압구정본점', '19882607', 4.36, 602, 178, '2022-08-23 00:00:00')]
[('크리스탈제이드 상하이 팰리스 잠실 에비뉴엘점', '35831858', 4.29, 688, 584, '2022-08-23 00:00:00')]
[('크리스탈제이드 현대백화점 천호점', '1655422043', 0, 36, 31, '2022-08-23 00:00:00')]
[('크리스탈제이드 AK 분당점', '1236473153', 4.29, 300, 113, '2022-08-23 00:00:00')]
[('크리스탈제이드 신세계 대구점', '48175941', 4.44, 929, 404, '2022-08-23 00:00:00')]
[('크리스탈제이드 롯데 부산서면점', '1371827794', 4.54, 582, 217, '2022-08-23 00:00:00')]
[('크리스탈제이드 신세계센텀시티몰점', '38230223', 4.29, 595, 297, '2022-08-23 00:00:00')]
[('크리스탈제이드 부산 피아크점', '1941012826', 0, 104, 113, '2022-08-23 00:00:00')]
[('더 키친 일뽀르노 광화문점', '18113

#### 🆗제품 리뷰 수집(API)

In [ ]:
# main.py
import pandas as pd
from datetime import datetime, timedelta
import requests
import time 
import re
from google.cloud import bigquery
from bs4 import BeautifulSoup

# 저장 위치 설정
# client = bigquery.Client('boxwood-chassis-356100')
# dataset_id = 'review'
# table_id = 'dinning_review'
# table_ref = client.dataset(dataset_id).table(table_id)
# Table = client.get_table(table_ref)  # API request

code = ['1271525551', '31378239',   '1110186348', '1141410734', '36220114',   '19882607', 
        '35831858',   '1655422043', '1236473153', '48175941',   '1371827794', '38230223', '1941012826',
        '1811388034', '1750908039', '1129408372', '1613879707', '1235447371', '1027299193'
        ]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}

def get_naver_summary(t1):
    date_   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    date_   = (datetime.today()+ timedelta(hours=9) - timedelta(days=0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    for store_code in code:
        url_temp2 = 'https://pcmap.place.naver.com/restaurant/'+str(store_code)+'/home'
        response = requests.get(url_temp2, headers = headers)
        response.encoding = 'utf-8'
        bsObj = BeautifulSoup(response.text, "html.parser")
        store_nm = bsObj('div', {'class' : 'place_didmount'})[0]('h1')[0].text
        # review_score = 0
        try:
            review_score = re.findall("별점</span><em>(.*?)</em>/5", str(bsObj('div', {'class' : 'place_didmount'})[0]))[0]
        except:
            review_score = 0
        review_count = bsObj('div', {'class' : 'place_didmount'})[0]('a', {'class' : 'place_bluelink'})[0]('em')[0].text.replace(',', '')
        blog_count   = bsObj('div', {'class' : 'place_didmount'})[0]('a', {'class' : 'place_bluelink'})[1]('em')[0].text.replace(',', '')
        rows_to_insert = [( str(store_code), store_nm, float(review_score), int(review_count), int(blog_count), date_)]
        print(rows_to_insert)
        # client.insert_rows(Table, rows_to_insert)
    return f'Hello World!'

In [ ]:
get_naver_summary('')

[('1271525551', '크리스탈제이드 용산 아이파크몰점', 4.32, 1342, 366, '2022-09-01 00:00:00')]
[('31378239', '크리스탈제이드 소공점', 4.2, 986, 562, '2022-09-01 00:00:00')]
[('1110186348', '크리스탈제이드 광화문상하이팰리스점', 4.3, 689, 140, '2022-09-01 00:00:00')]
[('1141410734', '크리스탈제이드 상하이 팰리스 잠원점', 4.43, 376, 134, '2022-09-01 00:00:00')]
[('36220114', '크리스탈제이드 파미에스테이션점', 4.3, 1076, 579, '2022-09-01 00:00:00')]
[('19882607', '크리스탈제이드 현대 압구정본점', 4.36, 608, 178, '2022-09-01 00:00:00')]
[('35831858', '크리스탈제이드 상하이 팰리스 잠실 에비뉴엘점', 4.29, 697, 586, '2022-09-01 00:00:00')]
[('1655422043', '크리스탈제이드 현대백화점 천호점', 0.0, 37, 30, '2022-09-01 00:00:00')]
[('1236473153', '크리스탈제이드 AK 분당점', 4.29, 301, 114, '2022-09-01 00:00:00')]
[('48175941', '크리스탈제이드 신세계 대구점', 4.44, 951, 409, '2022-09-01 00:00:00')]
[('1371827794', '크리스탈제이드 롯데 부산서면점', 4.54, 582, 218, '2022-09-01 00:00:00')]
[('38230223', '크리스탈제이드 신세계센텀시티몰점', 4.29, 600, 302, '2022-09-01 00:00:00')]
[('1941012826', '크리스탈제이드 부산 피아크점', 0.0, 111, 119, '2022-09-01 00:00:00')]
[('1811388034', '더 키친 

'Hello World!'